### Import Libraries

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Define Dataset Structure and Parameters

In [3]:
root_dir = "ECG_Image_Data"
subfolders = ["train", "test"]
image_size = (224, 224)  # Resizing dimensions
batch_size = 32

output_dir = "Extracted_Features"
os.makedirs(output_dir, exist_ok=True)

### Function to Extract Features

In [4]:
def extract_features(model, preprocess_func, model_name):
    datagen = ImageDataGenerator(preprocessing_function=preprocess_func)  
    
    for subfolder in subfolders:
        features = []
        labels = []
        
        generator = datagen.flow_from_directory(
            directory=os.path.join(root_dir, subfolder),
            target_size=image_size,
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False 
        )
        
        for batch_images, batch_labels in generator:
            batch_features = model.predict(batch_images)
            features.append(batch_features)
            labels.append(batch_labels)
        
        features = np.vstack(features)
        labels = np.vstack(labels)
        np.save(os.path.join(output_dir, f"{model_name}_{subfolder}_features.npy"), features)
        np.save(os.path.join(output_dir, f"{model_name}_{subfolder}_labels.npy"), labels)
        print(f"Features for {subfolder} saved using {model_name}.")

### Define Pre-trained Models

In [5]:
models = {
    "ResNet50": (ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3)), preprocess_resnet),
    "VGG16": (VGG16(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3)), preprocess_vgg),
    "MobileNetV2": (MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3)), preprocess_mobilenet),
    "InceptionV3": (InceptionV3(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3)), preprocess_inception)
}

87910968/87910968 [==============================] - 152s 2us/step


### Run Feature Extraction

In [ ]:
if __name__ == "__main__":
    for model_name, (model, preprocess_func) in models.items():
        print(f"Extracting features using {model_name}...")
        extract_features(model, preprocess_func, model_name)
    print("Feature extraction completed!")


Extracting features using ResNet50...
Found 99391 images belonging to 6 classes.
1/1 [==============================] - 19s 19s/step
